In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install --no-cache-dir \
  torch==2.5.0+cu124 \
  torchvision==0.20.0+cu124 \
  torchaudio==2.5.0+cu124 \
  --extra-index-url https://download.pytorch.org/whl/cu124
!pip install \
  https://download.pytorch.org/whl/cu124/xformers-0.0.29.post3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install \
  "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git" \
  git+https://github.com/unslothai/unsloth_zoo.git
!pip install \
  transformers==4.53.2 \
  trl==0.20.0 \
  bitsandbytes==0.46.1 \
  accelerate==1.4.0 \
  datasets==3.6.0 \
  huggingface_hub==0.34.3 \
  tyro==0.9.26 \
  cut_cross_entropy==25.1.1 \
  msgspec==0.19.0

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 280.5 MB/s  0:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 431.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 373.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 389.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 217.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 223.2 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 397.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 360.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.9 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 270.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#torch dynamo disapling and importing dependencies
import torch._dynamo
torch._dynamo.config.disable = True

import unsloth
import transformers
from transformers import TrainingArguments

print(transformers.__version__)
import torch

from unsloth.trainer import SFTTrainer
from peft import LoraConfig
import pandas as pd
from datasets import Dataset

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
4.53.2


In [ ]:
model_id = "google/gemma-3n-e2b-it"

#loading and getting model and tokenizer
model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.8.1: Fast Gemma3N patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [ ]:
# 2. Data Loading and Conversion to Dataset objects (remains the same)
train_df = pd.read_csv("dataset_train.csv")
eval_df = pd.read_csv("dataset_val.csv")

train_list = [{"query": q, "response": r} for q, r in zip(train_df["query"], train_df["response"])]
eval_list = [{"query": q, "response": r} for q, r in zip(eval_df["query"], eval_df["response"])]

train_data = Dataset.from_list(train_list)
eval_data = Dataset.from_list(eval_list)

In [ ]:
# Applying LoRA using Unsloth's integrated method (remains the same)
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=32,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"],
    lora_dropout=0.05,
    bias="none",
)

In [ ]:
#getting peft model
model = unsloth.FastLanguageModel.get_peft_model(
    model,
    r = lora_config.r,
    target_modules = list(lora_config.target_modules), # <--- Convert to list here
    lora_alpha = lora_config.lora_alpha,
    lora_dropout = lora_config.lora_dropout,
    bias = lora_config.bias,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
# 4. Training Arguments
training_args = TrainingArguments(
    output_dir="gemma3n-finetune-checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    report_to="none",
    torch_compile=False,
)


In [ ]:
#formatting_prompts_func for formatting inputs and response
def formatting_prompts_func(examples):
    instructions = examples["query"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = f"### Instruction:\n{instruction}\n\n### Response:\n{output}{tokenizer.eos_token}"
        texts.append(text)
    return texts

In [ ]:
# 6. SFTTrainer Initialization
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
    data_collator=None,
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/2807 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/366 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,807 | Num Epochs = 1 | Total steps = 351
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 45,809,664 of 5,485,247,936 (0.84% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,1.780300,2.093884
200,1.643600,1.989985
300,1.547300,1.854147


Unsloth: Not an error, but Gemma3nForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=351, training_loss=2.097850820957086, metrics={'train_runtime': 2011.9733, 'train_samples_per_second': 1.395, 'train_steps_per_second': 0.174, 'total_flos': 4110308507991168.0, 'train_loss': 2.097850820957086})